<a href="https://colab.research.google.com/github/advadj67/hw3_M11221004/blob/main/HW03_YoloV8_Colab_0519.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics
!git clone https://github.com/advadj67/hw3_M11221004.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.0/757.0 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
'''
#將.xml轉成Yolo所需 .txt檔案
import os
import xml.etree.ElementTree as ET

def convert_coordinates(size, box):
    # 轉換坐標
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_xml_to_yolo(xml_path, output_root, class_dict, output_folders):
    # 解析XML並轉換為YOLO格式
    tree = ET.parse(xml_path)
    root = tree.getroot()

    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for output_folder in output_folders:
        folder_name = os.path.basename(output_folder)
        output_dir = os.path.join(output_root, folder_name)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        output_file_path = os.path.join(output_dir, os.path.splitext(os.path.basename(xml_path))[0] + '.txt')
        with open(output_file_path, 'w') as f:
            for obj in root.findall('object'):
                cls = obj.find('name').text
                if cls not in class_dict:
                    continue
                cls_id = class_dict[cls]
                xml_box = obj.find('bndbox')
                box = (float(xml_box.find('xmin').text), float(xml_box.find('xmax').text),
                       float(xml_box.find('ymin').text), float(xml_box.find('ymax').text))
                bb = convert_coordinates((w,h), box)
                f.write(f"{cls_id} {' '.join([str(a) for a in bb])}\n")

# 資料夾列表
folders = ["訓練集_xml", "驗證集_xml", "測試集_xml"]
o_folders = ["train", "val", "test"]

# 分類字典，將類別名映射到整數標籤
class_dict = {"container": 0}

# 輸出根資料夾路徑
output_root = "E:\Downloads\貨櫃資料集\labels"

# 如果輸出根資料夾不存在，則創建
if not os.path.exists(output_root):
    os.makedirs(output_root)

# 迴圈處理每個資料夾
for folder, o_folder in zip(folders, o_folders):
    folder_path = os.path.join("E:\Downloads\貨櫃資料集", folder)  # 資料夾路徑
    xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]  # 獲取所有XML檔案
    output_folders = [os.path.join(output_root, o_folder)]  # 輸出資料夾路徑
    for xml_file in xml_files:
        xml_path = os.path.join(folder_path, xml_file)  # XML檔案路徑
        convert_xml_to_yolo(xml_path, output_root, class_dict, output_folders)

print("labels轉換完成!")
'''

'\n#將.xml轉成Yolo所需 .txt檔案\nimport os\nimport xml.etree.ElementTree as ET\n\ndef convert_coordinates(size, box):\n    # 轉換坐標\n    dw = 1.0 / size[0]\n    dh = 1.0 / size[1]\n    x = (box[0] + box[1]) / 2.0\n    y = (box[2] + box[3]) / 2.0\n    w = box[1] - box[0]\n    h = box[3] - box[2]\n    x = x * dw\n    w = w * dw\n    y = y * dh\n    h = h * dh\n    return (x, y, w, h)\n\ndef convert_xml_to_yolo(xml_path, output_root, class_dict, output_folders):\n    # 解析XML並轉換為YOLO格式\n    tree = ET.parse(xml_path)\n    root = tree.getroot()\n\n    size = root.find(\'size\')\n    w = int(size.find(\'width\').text)\n    h = int(size.find(\'height\').text)\n\n    for output_folder in output_folders:\n        folder_name = os.path.basename(output_folder)\n        output_dir = os.path.join(output_root, folder_name)\n        if not os.path.exists(output_dir):\n            os.makedirs(output_dir)\n\n        output_file_path = os.path.join(output_dir, os.path.splitext(os.path.basename(xml_path))[0] + \'.

In [ ]:
'''
from PIL import Image

# 資料夾列表
folder_path = "E:/Downloads/貨櫃資料集/"
folders = [folder_path + "訓練集", folder_path + "驗證集", folder_path + "測試集"]

output_folders = [folder_path + "images/train", folder_path + "images/val", folder_path + "images/test"]


# 創建輸出資料夾
for output_folder in output_folders:
    os.makedirs(output_folder, exist_ok=True)

# 轉換函數
def resize_images(folder, output_folder):
    # 獲取資料夾中所有圖片的檔案名稱
    files = os.listdir(folder)

    # 迴圈處理每張圖片
    for file in files:
        # 檔案路徑
        file_path = os.path.join(folder, file)

        # 如果是檔案
        if os.path.isfile(file_path):
            # 打開圖片
            img = Image.open(file_path)

            # 重新調整大小
            resized_img = img.resize((416, 416))

            # 另存新圖片
            resized_img.save(os.path.join(output_folder, file))

# 對每個資料夾和對應的輸出資料夾調用resize_images函數
for folder, output_folder in zip(folders, output_folders):
    resize_images(folder, output_folder)

print("images轉換完成!")
'''

'\nfrom PIL import Image\n\n# 資料夾列表\nfolder_path = "E:/Downloads/貨櫃資料集/"\nfolders = [folder_path + "訓練集", folder_path + "驗證集", folder_path + "測試集"]\n\noutput_folders = [folder_path + "images/train", folder_path + "images/val", folder_path + "images/test"]\n\n\n# 創建輸出資料夾\nfor output_folder in output_folders:\n    os.makedirs(output_folder, exist_ok=True)\n\n# 轉換函數\ndef resize_images(folder, output_folder):\n    # 獲取資料夾中所有圖片的檔案名稱\n    files = os.listdir(folder)\n\n    # 迴圈處理每張圖片\n    for file in files:\n        # 檔案路徑\n        file_path = os.path.join(folder, file)\n\n        # 如果是檔案\n        if os.path.isfile(file_path):\n            # 打開圖片\n            img = Image.open(file_path)\n\n            # 重新調整大小\n            resized_img = img.resize((416, 416))\n\n            # 另存新圖片\n            resized_img.save(os.path.join(output_folder, file))\n\n# 對每個資料夾和對應的輸出資料夾調用resize_images函數\nfor folder, output_folder in zip(folders, output_folders):\n    resize_images(folder, output_folder)\n\nprint(

In [ ]:
import shutil

# 要刪除的資料夾路徑
folder_path = '/content/runs/detect'
folder_path2 = '/content/pre_results'
# 使用 shutil.rmtree() 函數刪除資料夾
#shutil.rmtree(folder_path)
shutil.rmtree(folder_path2)

In [2]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data='/content/hw3_M11221004/貨櫃資料集/data.yaml', epochs=10, batch=8, imgsz=416)

model.val()  # It'll automatically evaluate the data you trained.

100%|██████████| 6.23M/6.23M [00:00<00:00, 148MB/s]

Transferred 355/355 items from pretrained weights


Ultralytics YOLOv8.2.17 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/hw3_M11221004/貨櫃資料集/data.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

100%|██████████| 755k/755k [00:00<00:00, 65.1MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:07<00:00,  4.52it/s]


                   all        536        536          1          1      0.995      0.872
Speed: 0.3ms preprocess, 3.0ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train
Ultralytics YOLOv8.2.17 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/hw3_M11221004/貨櫃資料集/labels/val.cache... 536 images, 0 backgrounds, 0 corrupt: 100%|██████████| 536/536 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:05<00:00, 12.19it/s]


                   all        536        536          1          1      0.995      0.872
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7805ebd3f550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [3]:
import os

# 定義圖片檔案目錄路徑
source_directory = '/content/hw3_M11221004/貨櫃資料集/images/test/'

# 指定儲存目錄
save_directory = '/content/predictions/'

# 預測整個目錄
results = model.predict(source=source_directory, save=True)

# 轉換預測框格式
pred_boxes_for_iou = []
for result in results:
    # Access the path attribute from the Result object
    image_path = result.path
    image_id = os.path.splitext(os.path.basename(image_path))[0]
    boxes = [{'class_index': box.cls, 'coordinates': box.xyxy} for box in result.boxes]
    pred_boxes_for_iou.append({'image_id': image_id, 'boxes': boxes})

# 將預測結果保存到文件以便後續使用
import pickle
with open('pred_boxes_for_iou.pkl', 'wb') as f:
    pickle.dump(pred_boxes_for_iou, f)

print("預測框完成")



image 1/755 /content/hw3_M11221004/貨櫃資料集/images/test/image_0001.jpg: 416x416 1 container number, 12.3ms
image 2/755 /content/hw3_M11221004/貨櫃資料集/images/test/image_0002.jpg: 416x416 1 container number, 17.4ms
image 3/755 /content/hw3_M11221004/貨櫃資料集/images/test/image_0003.jpg: 416x416 1 container number, 11.8ms
image 4/755 /content/hw3_M11221004/貨櫃資料集/images/test/image_0004.jpg: 416x416 1 container number, 17.6ms
image 5/755 /content/hw3_M11221004/貨櫃資料集/images/test/image_0005.jpg: 416x416 1 container number, 12.6ms
image 6/755 /content/hw3_M11221004/貨櫃資料集/images/test/image_0006.jpg: 416x416 1 container number, 14.7ms
image 7/755 /content/hw3_M11221004/貨櫃資料集/images/test/image_0007.jpg: 416x416 1 container number, 13.1ms
image 8/755 /content/hw3_M11221004/貨櫃資料集/images/test/image_0008.jpg: 416x416 1 container number, 12.9ms
image 9/755 /content/hw3_M11221004/貨櫃資料集/images/test/image_0009.jpg: 416x416 1 container number, 12.9ms
image 10/755 /content/hw3_M11221004/貨櫃資料集/images/test/image_001

In [4]:
import os
image_width = image_height = 416

def read_true_boxes(labels_directory):
    true_boxes = {}
    for label_file in os.listdir(labels_directory):
        if label_file.endswith('.txt'):
            with open(os.path.join(labels_directory, label_file), 'r') as f:
                lines = f.readlines()
            boxes = []
            for line in lines:
                parts = line.strip().split()
                class_index = int(parts[0])
                x_center = float(parts[1])
                y_center = float(parts[2])
                width = float(parts[3])
                height = float(parts[4])
                # 計算左上角和右下角坐標
                x1 = int((x_center - width / 2) * image_width)
                y1 = int((y_center - height / 2) * image_height)
                x2 = int((x_center + width / 2) * image_width)
                y2 = int((y_center + height / 2) * image_height)
                boxes.append({'class_index': class_index, 'coordinates': [x1, y1, x2, y2]})
            image_id = os.path.splitext(label_file)[0]
            true_boxes[image_id] = boxes
    return true_boxes

# 真實框的資料夾路徑
labels_directory = '/content/hw3_M11221004/貨櫃資料集/labels/test'

# 讀取真實框
true_boxes = read_true_boxes(labels_directory)

# 將真實框轉換成易於計算 IoU 的格式
true_boxes_for_iou = [{'image_id': image_id, 'boxes': boxes} for image_id, boxes in true_boxes.items()]

# 將真實框保存到文件以便後續使用
import pickle
with open('true_boxes_for_iou.pkl', 'wb') as f:
    pickle.dump(true_boxes_for_iou, f)




print("真實框轉換完成")


真實框轉換完成


In [25]:
print("Predicted coordinates:", pred_coordinates)
print("True coordinates:", true_coordinates)


Predicted coordinates: tensor([[163.1823,  43.1231, 239.5382,  88.3062]], device='cuda:0')
True coordinates: [161, 43, 240, 91]


In [30]:
# 列印第一個真實框的資訊
for true_box in true_boxes:
    print("True box for image", true_box['image_id'], ":")
    if true_box['boxes']:
        box = true_box['boxes'][0]
        x_min = box['coordinates'][0] - box['coordinates'][2] / 2
        y_min = box['coordinates'][1] - box['coordinates'][3] / 2
        x_max = box['coordinates'][0] + box['coordinates'][2] / 2
        y_max = box['coordinates'][1] + box['coordinates'][3] / 2
        print("Class Index:", box['class_index'], "Coordinates:", [x_min, y_min, x_max, y_max])

# 列印第一個預測框的資訊
for prediction in predictions:
    print("Predicted box for image", prediction['image_id'], ":")
    if prediction['boxes']:
        box = prediction['boxes'][0]
        class_index = int(box['class_index'].item())  # 提取標量值
        x_min = box['coordinates'][0] - box['coordinates'][2] / 2
        y_min = box['coordinates'][1] - box['coordinates'][3] / 2
        x_max = box['coordinates'][0] + box['coordinates'][2] / 2
        y_max = box['coordinates'][1] + box['coordinates'][3] / 2
        print("Class Index:", class_index, "Coordinates:", [x_min, y_min, x_max, y_max])



True box for image image_0460 :
Class Index: 0 Coordinates: [-93.5, -63.25, 195.5, 136.25]
True box for image image_0160 :
Class Index: 0 Coordinates: [-79.5, -25.75, 205.5, 8.75]
True box for image image_0575 :
Class Index: 0 Coordinates: [-85.25, -46.5, 250.25, 100.5]
True box for image image_0578 :
Class Index: 0 Coordinates: [-102.25, -67.0, 235.25, 151.0]
True box for image image_0692 :
Class Index: 0 Coordinates: [-108.25, -84.75, 201.25, 191.75]
True box for image image_0254 :
Class Index: 0 Coordinates: [-86.25, -26.5, 231.25, 8.5]
True box for image image_0421 :
Class Index: 0 Coordinates: [-70.0, -33.0, 226.0, 73.0]
True box for image image_0302 :
Class Index: 0 Coordinates: [-70.75, -22.25, 193.75, -0.75]
True box for image image_0556 :
Class Index: 0 Coordinates: [-82.5, -46.0, 240.5, 98.0]
True box for image image_0280 :
Class Index: 0 Coordinates: [-84.75, -32.0, 211.75, 16.0]
True box for image image_0400 :
Class Index: 0 Coordinates: [-97.5, -71.0, 219.5, 139.0]
True bo

In [31]:
def calculate_iou(box1, box2):
    # 計算交集的左上角和右下角坐標
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    # 計算交集區域的面積
    intersection_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # 計算兩個框的面積
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # 計算 IoU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)

    return iou


In [49]:
TP = 0
FP = 0
FN = 0
total_iou = 0
total_predictions = 0
iou_threshold = 0.7 #IoU 大於 0.7 才算正樣本

for prediction in predictions:
    image_id = prediction['image_id']
    pred_boxes = prediction['boxes']

    true_boxes_for_image = [box['boxes'] for box in true_boxes if box['image_id'] == image_id][0]
    true_boxes_coordinates = [box['coordinates'] for box in true_boxes_for_image]

    matched_true_boxes = set()

    for pred_box in pred_boxes:
        pred_coordinates = pred_box['coordinates']
        max_iou = 0
        is_TP = False

        for true_box_index, true_coordinates in enumerate(true_boxes_coordinates):
            iou = calculate_iou(pred_coordinates, true_coordinates)
            max_iou = max(max_iou, iou)

            if iou >= iou_threshold and true_box_index not in matched_true_boxes:
                TP += 1
                is_TP = True
                matched_true_boxes.add(true_box_index)
                break

        total_iou += max_iou
        if not is_TP:
            FP += 1

    total_predictions += len(pred_boxes)
    FN += len(true_boxes_coordinates) - len(matched_true_boxes)

# 計算 Precision、Recall、F1-score、mAP
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
mAP = total_iou / total_predictions

# 打印所有指標的值
print("TP:", TP)
print("FP:", FP)
print("FN:", FN)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("mAP:", mAP)


TP: 755
FP: 1
FN: 0
Precision: 0.9986772486772487
Recall: 1.0
F1 Score: 0.99933818663137
mAP: 0.9611289975579611


import shutil

# 要删除的文件夹路径
folder_path = '/content/pre_results'

# 尝试删除文件夹，如果文件夹不存在则捕获异常
try:
    shutil.rmtree(folder_path)
    print(f'Folder {folder_path} has been deleted.')
except FileNotFoundError:
    print(f'Folder {folder_path} does not exist, skipping deletion.')

In [ ]:
import shutil

# 要删除的文件夹路径
folder_path = '/content/pre_results'

# 尝试删除文件夹，如果文件夹不存在则捕获异常
try:
    shutil.rmtree(folder_path)
    print(f'Folder {folder_path} has been deleted.')
except FileNotFoundError:
    print(f'Folder {folder_path} does not exist, skipping deletion.')


import os
import cv2
import numpy as np

def apply_nms(boxes, scores, threshold):
    indices = cv2.dnn.NMSBoxes(boxes, scores, score_threshold=threshold, nms_threshold=0.4)
    return [boxes[idx[0]] for idx in indices] if indices else []

source_folder = '/content/hw3_M11221004/貨櫃資料集/images/test'
output_folder = '/content/pre_results'
labels_folder = '/content/hw3_M11221004/貨櫃資料集/labels/test'

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(source_folder):
    if filename.endswith(('.jpg', '.png')):
        source_path = os.path.join(source_folder, filename)
        output_path = os.path.join(output_folder, filename)
        label_path = os.path.join(labels_folder, os.path.splitext(filename)[0] + '.txt')

        # 进行预测
        results = model(source_path)

        img = cv2.imread(source_path)

        # 收集预测结果
        pred_boxes = [box.xyxy.cpu().numpy().flatten()[:4] for box in results[0].boxes]
        pred_scores = [box.conf.cpu().numpy().item() for box in results[0].boxes]

        # 应用 NMS，保留一个框
        filtered_boxes = apply_nms(pred_boxes, pred_scores, threshold=0.5)

        # 如果过滤后的预测框为空，则选择最高置信度的框
        if not filtered_boxes:
            max_score_idx = np.argmax(pred_scores)
            filtered_boxes = [pred_boxes[max_score_idx]]

        print("Number of filtered boxes:", len(filtered_boxes))

        # 绘制预测框并保存图片
        for box in filtered_boxes:
            x1, y1, x2, y2 = box
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

        cv2.imwrite(output_path, img)


Folder /content/pre_results has been deleted.

image 1/1 /content/hw3_M11221004/貨櫃資料集/images/test/image_0667.jpg: 416x416 1 container number, 9.7ms
Speed: 1.1ms preprocess, 9.7ms inference, 1.6ms postprocess per image at shape (1, 3, 416, 416)
Number of filtered boxes: 1

image 1/1 /content/hw3_M11221004/貨櫃資料集/images/test/image_0017.jpg: 416x416 1 container number, 12.9ms
Speed: 0.9ms preprocess, 12.9ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 416)
Number of filtered boxes: 1

image 1/1 /content/hw3_M11221004/貨櫃資料集/images/test/image_0168.jpg: 416x416 1 container number, 9.3ms
Speed: 0.9ms preprocess, 9.3ms inference, 1.6ms postprocess per image at shape (1, 3, 416, 416)
Number of filtered boxes: 1

image 1/1 /content/hw3_M11221004/貨櫃資料集/images/test/image_0605.jpg: 416x416 1 container number, 9.4ms
Speed: 0.9ms preprocess, 9.4ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 416)
Number of filtered boxes: 1

image 1/1 /content/hw3_M11221004/貨櫃資料集/images/t

## 文字辨識OCR

In [ ]:
!sudo apt-get update
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev
!pip install pytesseract

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,383 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,118 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/mai

In [ ]:
import cv2
import pytesseract
from google.colab.patches import cv2_imshow

# 定義裁切並進行文字辨識的函數
def crop_and_recognize(image, bounding_box):
    # 裁剪框框的區域
    x1, y1, x2, y2 = map(int, bounding_box)
    cropped_image = image[y1:y2, x1:x2]
    cv2_imshow(cropped_image)
    # 將裁剪後的圖片轉換為灰階
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)

    # 使用大津法計算閾值
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # 反轉二值化圖片，將背景設置為白色，文字設置為黑色
    inverted_image = cv2.bitwise_not(binary_image)

    # 設置文字辨識引擎（使用Tesseract OCR）
    pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

    # 進行文字辨識
    text = pytesseract.image_to_string(inverted_image)

    # 顯示裁切後的圖片
    cv2_imshow(inverted_image)

    # 顯示辨識的結果
    print("Text in box:", text)

# 要處理的圖片路徑
image_path = '/content/pre_results/image_0001.jpg'

# 讀取圖片
image = cv2.imread(image_path)

# 單個框框的座標
bounding_box = results[0]

# 裁切並進行文字辨識
crop_and_recognize(image, bounding_box)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Results'

### **缺乏：**


1. mAP、Recall、Precision和F1-score
2. 準確率績效公式: 辨識正確筆數/測試資料筆數
3. 車牌偵測及文字辨識
4. 影片的








**遇到UTF-8錯誤**

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

**下載資料夾**

In [ ]:
#壓縮成資料夾下載，前面是檔名.zip，後面是要被壓縮的東西
!zip -r /content/pred_1.zip /content/runs

from google.colab import files
files.download("/content/pred_1.zip")

  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/train2/ (stored 0%)
  adding: content/runs/detect/train2/val_batch1_pred.jpg (deflated 11%)
  adding: content/runs/detect/train2/confusion_matrix.png (deflated 37%)
  adding: content/runs/detect/train2/val_batch2_labels.jpg (deflated 12%)
  adding: content/runs/detect/train2/val_batch1_labels.jpg (deflated 11%)
  adding: content/runs/detect/train2/PR_curve.png (deflated 29%)
  adding: content/runs/detect/train2/confusion_matrix_normalized.png (deflated 36%)
  adding: content/runs/detect/train2/val_batch0_labels.jpg (deflated 12%)
  adding: content/runs/detect/train2/R_curve.png (deflated 20%)
  adding: content/runs/detect/train2/val_batch2_pred.jpg (deflated 11%)
  adding: content/runs/detect/train2/val_batch0_pred.jpg (deflated 11%)
  adding: content/runs/detect/train2/F1_curve.png (deflated 19%)
  adding: content/runs/detect/train2/P_curve.png (deflated 26%)
  adding: content/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>